# Codificación de la fuente
<p><code>Python en Jupyter Notebook</code></p>
<p>Creado por <code>Giancarlo Ortiz</code> para el curso de <code>Redes</code></p>

## La fuente
Una fuente es todo aquello que emite mensajes; puede entenderse como un dispositivo de transmisión de datos, cuyos mensajes son los datos enviados.

### Agenda
1. Tipos de fuente
1. Tipos de código
1. Codificación de fuentes


In [2]:
# Importar módulos al cuaderno
import heapq
import math as m
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import stats as st


# Definir e incluir nuevas funciones al cuaderno
def make_tree(probabilities: dict) -> tuple:
    heap = []
    # Agregamos todos los símbolos al montículo (árbol)
    # Este montículo del modulo heapd estará siempre organizado por mínimos
    for symbol, potluck in probabilities.items():
        # La fila de prioridad está ordenada por
        # prioridad y profundidad
        heapq.heappush(heap, (potluck, 0, symbol))

    # Empezamos a mezclar símbolos juntos
    # hasta que la fila tenga un elemento
    while len(heap) > 1:
        # Desapila los símbolos de prioridad mas baja
        e1 = heapq.heappop(heap)
        e2 = heapq.heappop(heap)
        # Apila un nuevo elemento
        # Este nuevo nodo tiene probabilidad e1[0]+e2[0]
        # y profundidad mayor al nuevo nodo
        nw_e = (e1[0]+e2[0], max(e1[1], e2[1])+1, [e1, e2])
        heapq.heappush(heap, nw_e)

    # Devolvemos el arbol sin la fila
    return heap[0]


def make_dictionary(tree: tuple) -> dict:
    res = {}  # La estructura que vamos a devolver
    search_stack = []  # Pila para DFS
    # El último elemento de la lista es el prefijo!
    search_stack.append(tree+("",))
    while len(search_stack) > 0:
        elm = search_stack.pop()
        if type(elm[2]) == list:
            # En este caso, el nodo NO es una hoja del árbol,
            # es decir que tiene nodos hijos
            prefix = elm[-1]
            # El hijo izquierdo tiene "0" en el prefijo
            search_stack.append(elm[2][1]+(prefix+"0",))
            # El hijo derecho tiene "1" en el prefijo
            search_stack.append(elm[2][0]+(prefix+"1",))
            continue
        else:
            # El nodo es una hoja del árbol, así que
            # obtenemos el código completo y lo agregamos
            code = elm[-1]
            res[elm[2]] = code
        pass
    return res


## 1. Tipos de fuente
---
### 1.1. Fuente aleatoria:
Según la naturaleza generativa de sus mensajes, una fuente puede ser determinista o [aleatoria](https://es.wikipedia.org/wiki/Variable_aleatoria).

### 1.2. Fuente discreta:
Según los valores que de sus mensajes, una fuente puede ser continua o [discreta](https://es.wikipedia.org/wiki/Funci%C3%B3n_discreta).

### 1.3. Fuente estructurada
Según la relación entre sus mensajes, una fuente puede ser caótica o [estructurada](https://es.wikipedia.org/wiki/Variable_aleatoria).

## 2. Tipos de código
---
### 2.1. Código bloque:
Los códigos bloque no tienen memoria, la secuencia de salida depende únicamente de el símbolo de entrada.

### 2.2. Código No singular:
Una singularidad se produce cuando dos símbolos tienen el mismo código; en un código no singular todas sus palabras son distintas

| Fuente | Código singular | Código no singular |
|:--:|:--:|:--:|
| $ S_0 $ |  0 |  0 |
| $ S_1 $ | __00__ | 00 |
| $ S_2 $ | 01 | 01 |
| $ S_3 $ | __00__ | 10 |


### 2.3. Código univoco:
En un código univoco la secuencia de salida es única para $E^n$.

| Fuente | Código no univoco | Código univoco |
|:--:|:--:|:--:|
| $ S_0 $ |  __0__ |  11 |
| $ S_1 $ | __00__ | 00 |
| $ S_2 $ | 01 | 01 |
| $ S_3 $ | 10 | 10 |

### 2.3. Código instantáneo:.
En un código instantáneo Se decodifica la palabra al momento que llega, sin esperar la siguiente palabra.

<code>Ejemplo:</code> carácter de inicio vs carácter de fin.

| Fuente | Código no instantáneo | Código instantáneo |
|:--:|:--|--:|
| $ S_0 $ | __0__ |  __0__ |
| $ S_1 $ | __0__ 1 | 1 __0__ |
| $ S_2 $ | __0__ 11 | 11 __0__ |
| $ S_3 $ | __0__ 111 | 111 __0__ |



## 3. Codificación de la fuente
---
### 3.1. Definición:
Dada una fuente aleatoria y discreta con alfabeto (__FUENTE__) finito $F$ y dado un alfabeto (__DESTINO__) de codificación también finito D, se considerarán únicamente los códigos consistentes en asignar unívocamente a cada elemento de $F^n$ , para un valor de n dado, una secuencia de elementos de D, que denominaremos palabra del código.

Se basa en:
Elementos mas largos tienen 

### 3.2. Obtener el árbol de Huffman:
Para obtener el árbol de prioridad según el [algoritmo de Huffman](https://es.wikipedia.org/wiki/Algoritmo_de_Huffman):
1. Poner todos los símbolos en una fila de prioridad de acuerdo a sus probabilidades.
1. Combinar los dos símbolos menos probables en un solo nodo de un árbol.
1. Insertar el nuevo nodo a la fila de prioridad
1. Repetir desde el paso 2 hasta que sólo haya un elemento en la fila de prioridad.

### 3.3. Obtener el código Huffman:
Para obtener el código asociado a un símbolo:
1. Comenzar con un código vacío
1. Iniciar el recorrido del árbol en la hoja asociada al símbolo
1. Comenzar un recorrido del árbol hacia arriba
1. Cada vez que se suba un nivel, añadir al código la etiqueta de la rama que se ha recorrido
1. Tras llegar a la raíz, invertir el código
1. El resultado es el código Huffman deseado

### <code>Ejemplo:</code> Alfabeto español
Si se desea transmitir un mensaje binario escrito originalmente usando las mayúsculas del alfabeto latino y según la [frecuencia de aparición](https://es.wikipedia.org/wiki/Frecuencia_de_aparici%C3%B3n_de_letras) de letras en el español:

* Encuentre un código optimo para de símbolos del alfabeto latino en español.

In [16]:
# Matriz de probabilidades
probabilidades = {
    "A": 0.1253, "B": 0.0142, "C": 0.0468, "D": 0.0586, "E": 0.1368, "F": 0.0069,
    "G": 0.0101, "H": 0.0070, "I": 0.0625, "J": 0.0044, "K": 0.0001, "L": 0.0497,
    "M": 0.0315, "N": 0.0671, "Ñ": 0.0031, "O": 0.0868, "P": 0.0251, "Q": 0.0088,
    "R": 0.0687, "S": 0.0798, "T": 0.0463, "U": 0.0393, "V": 0.0090, "W": 0.0001,
    "X": 0.0022, "Y": 0.0090, "Z": 0.0052}

# Código Huffman
Huffman_code = make_dictionary(make_tree(probabilidades))

# Salida
print(f"|", "-"*38, "|")
print("| Símbolo | Probabilidad | Código optimo |")
print(f"|", "-"*38, "|")
for item in probabilidades.keys():
    print(
        f"| {item:^7s} | {probabilidades[item]:12} | {Huffman_code[item]:13} |")
print(f"|", "-"*38, "|")


| -------------------------------------- |
| Símbolo | Probabilidad | Código optimo |
| -------------------------------------- |
|    A    |       0.1253 | 100           |
|    B    |       0.0142 | 101110        |
|    C    |       0.0468 | 1101          |
|    D    |       0.0586 | 1010          |
|    E    |       0.1368 | 010           |
|    F    |       0.0069 | 1011110       |
|    G    |       0.0101 | 111110        |
|    H    |        0.007 | 0010111       |
|    I    |       0.0625 | 0111          |
|    J    |       0.0044 | 1111111       |
|    K    |       0.0001 | 1011111111    |
|    L    |       0.0497 | 1100          |
|    M    |       0.0315 | 10110         |
|    N    |       0.0671 | 0110          |
|    Ñ    |       0.0031 | 10111110      |
|    O    |       0.0868 | 0000          |
|    P    |       0.0251 | 11110         |
|    Q    |       0.0088 | 0010110       |
|    R    |       0.0687 | 0011          |
|    S    |       0.0798 | 0001          |
|    T    |

### <code>Ejemplo:</code> Alfabeto español
Si se desea optimizar la transmisión de un mensaje binario codificado desde una fuente aleatoria pero estructurada según la tabla a continuación:

* Encuentre un código que optimice el existente.
* Calcule la ganancia por cada 100K de datos transmitidos.

| Símbolo | Código no optimo | Probabilidad |
|:--:|:--:|:--:|
|1|000|0.20|
|2|001|0.01|
|3|010|0.40|
|4|011|0.04|
|5|100|0.10|
|6|101|0.02|
|7|110|0.07|
|8|111|0.16|


In [27]:
probabilidades = {
    "000": 0.2,
    "001": 0.01,
    "010": 0.4,
    "011": 0.04,
    "100": 0.1,
    "101": 0.02,
    "110": 0.07,
    "111": 0.16}

Huffman_tree = make_tree(probabilidades)
Huffman_code = make_dictionary(Huffman_tree)

suma = 0
for symbol, potluck in Huffman_code.items():
    suma += (3 - len(potluck)) * probabilidades[symbol]

enviados = suma*100
no_enviados = 100-enviados

# Salida
print(f"|", "-"*38, "|")
print("| Símbolo | Probabilidad | Código optimo |")
print(f"|", "-"*38, "|")
for item in probabilidades.keys():
    print(
        f"| {item:^7s} | {probabilidades[item]:12} | {Huffman_code[item]:13} |")
print(f"|", "-"*38, "|")
print(f"Enviados por cada 100K: {enviados:.0f}K")
print(f"Ahorro por cada 100K:   {no_enviados:.0f}K")


| -------------------------------------- |
| Símbolo | Probabilidad | Código optimo |
| -------------------------------------- |
|   000   |          0.2 | 000           |
|   001   |         0.01 | 010011        |
|   010   |          0.4 | 1             |
|   011   |         0.04 | 01000         |
|   100   |          0.1 | 011           |
|   101   |         0.02 | 010010        |
|   110   |         0.07 | 0101          |
|   111   |         0.16 | 001           |
| -------------------------------------- |
Enviados por cada 100K: 56K
Ahorro por cada 100K:   44K


---
## Mas Recursos

- [Codificación Huffman](https://es.wikipedia.org/wiki/Codificaci%C3%B3n_Huffman) (Wikipedia)
- [Codificación de canales con ruido](https://es.wikipedia.org/wiki/Teorema_de_Shannon-Hartley) (Wikipedia)
- [Codificación Huffman](https://es.wikipedia.org/wiki/Codificaci%C3%B3n_Huffman) (Wikipedia)
- [Capacidad de canal](https://es.wikipedia.org/wiki/Capacidad_de_canal) (Wikipedia)